In [1]:
import numpy as np
import subprocess
import random
from glob import glob
from skimage import io
from skimage.transform import rescale,resize,downscale_local_mean
import SimpleITK as sitk
import os
from matplotlib import pyplot as plt
from matplotlib import cm

In [2]:
#파일 읽어오기
file_name_image = '/Users/donghoon/공부자료/포스텍정보통신연구소연구부/공모전/sample/sample_cardiac/CHD/image/**.mha'
file_name_label = '/Users/donghoon/공부자료/포스텍정보통신연구소연구부/공모전/sample/sample_cardiac/CHD/label/**.mha'


In [3]:
# 파일리스트 정렬하기
file_list_image = sorted(glob(file_name_image))
file_list_label = sorted(glob(file_name_label))
print(file_list_image)
print(file_list_label)

['/Users/donghoon/공부자료/포스텍정보통신연구소연구부/공모전/sample/sample_cardiac/CHD/image/sample_dia.mha', '/Users/donghoon/공부자료/포스텍정보통신연구소연구부/공모전/sample/sample_cardiac/CHD/image/sample_sys.mha']
['/Users/donghoon/공부자료/포스텍정보통신연구소연구부/공모전/sample/sample_cardiac/CHD/label/sample_dia_M.mha', '/Users/donghoon/공부자료/포스텍정보통신연구소연구부/공모전/sample/sample_cardiac/CHD/label/sample_sys_M.mha']


In [4]:
#파일 인덱스 0 읽어오기 & 정보 보기
image = sitk.ReadImage(str(file_list_image[0]))
label = sitk.ReadImage(str(file_list_label[0]))
print(sitk.GetArrayFromImage(image).shape)
print(sitk.GetArrayFromImage(label).shape)
print(image.GetPixelIDTypeAsString())
print(label.GetPixelIDTypeAsString())
print(image.GetSpacing())
print(label.GetSpacing())

(171, 512, 512)
(171, 512, 512)
16-bit signed integer
8-bit unsigned integer
(0.21875, 0.21875, 0.400024)
(1.0, 1.0, 1.0)


In [5]:
# 변수 지정
dimension = 3
point = (1.0, 1.0,1.0)
LowT = -300
HighT = 700
displacement = 10
n_class =3
img_rows=512
img_cols=512

In [6]:
# 포인트 Transform 좌표 프린트하기
def transform_point(transform, point):
    transformed_point = transform.TransformPoint(point)
    print('Point '+str(point) + ' transformed is ' + str(transformed_point))

In [7]:
# 없어도 되는 코드
print('*Translation Transform*')
translation = sitk.TranslationTransform(dimension)

print('Parameters : '+ str(translation.GetParameters()))
print('Offset : '+ str(translation.GetOffset()))
print('FixedParameters: '+ str(translation.GetFixedParameters()))
transform_point(translation,point)

print('')
#translation.SetParameters((3.1,4.4,5.3))
print('Parameters : '+ str(translation.GetParameters()))
transform_point(translation, point)

*Translation Transform*
Parameters : (0.0, 0.0, 0.0)
Offset : (0.0, 0.0, 0.0)
FixedParameters: ()
Point (1.0, 1.0, 1.0) transformed is (1.0, 1.0, 1.0)

Parameters : (0.0, 0.0, 0.0)
Point (1.0, 1.0, 1.0) transformed is (1.0, 1.0, 1.0)


In [8]:
# 2D 이미지 보여주기
def myshow2d(img, title = None, margin=0.05, dpi=80):
    nda = sitk.GetArrayFromImage(img)
    spacing = img.GetSpacing()
    
    ysize = nda.shape[0]
    xsize = nda.shape[1]
    
    figsize = (1+margin) * ysize /dpi, (1+margin) * xsize /dpi
    
    fig = plt.figure(title, figsize = figsize , dpi = dpi)
    ax = fig.add_axes([margin,margin,1 - 2*margin, 1-2*margin])
    
    extent = (0, xsize*spacing[1], 0, ysize*spacing[0])
    
    t = ax.imshow(nda, extent=extent, cmap= 'gray', origin='lower')
    if(title):
        plt.title(title)
    plt.show()

In [9]:
# 2D 이미지 보여주기
def myshow(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayFromImage(img)
    spacing = img.GetSpacing()

    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]

        # the the number of components is 3 or 4 consider it an RGB image
        if c not in (3, 4):
            nda = nda[nda.shape[0] // 2, :, :]

    elif nda.ndim == 4:
        c = nda.shape[-1]

        if c not in (3, 4):
            raise RuntimeError("Unable to show 3D-vector Image")

        # take a z-slice
        nda = nda[nda.shape[0] // 2, :, :, :]

    xsize = nda.shape[1]
    ysize = nda.shape[0]

    # Make a figure big enough to accommodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * xsize / dpi, (1 + margin) * ysize / dpi

    plt.figure(figsize=figsize, dpi=dpi, tight_layout=True)
    ax = plt.gca()

    extent = (0, xsize * spacing[0], ysize * spacing[1], 0)

    t = ax.imshow(nda, extent=extent, interpolation=None)

    if nda.ndim == 2:
        t.set_cmap("gray")

    if(title):
        plt.title(title)

    plt.show()

# 3D 이미지 보여주기
def myshow3d(img, xslices=[], yslices=[], zslices=[], title=None, margin=0.05,
             dpi=80):
    img_xslices = [img[s, :, :] for s in xslices]
    img_yslices = [img[:, s, :] for s in yslices]
    img_zslices = [img[:, :, s] for s in zslices]

    maxlen = max(len(img_xslices), len(img_yslices), len(img_zslices))


    img_null = sitk.Image([0, 0], img.GetPixelID(),
                          img.GetNumberOfComponentsPerPixel())

    img_slices = []
    d = 0

    if len(img_xslices):
        img_slices += img_xslices + [img_null] * (maxlen - len(img_xslices))
        d += 1

    if len(img_yslices):
        img_slices += img_yslices + [img_null] * (maxlen - len(img_yslices))
        d += 1

    if len(img_zslices):
        img_slices += img_zslices + [img_null] * (maxlen - len(img_zslices))
        d += 1

    if maxlen != 0:
        if img.GetNumberOfComponentsPerPixel() == 1:
            img = sitk.Tile(img_slices, [maxlen, d])
        # TO DO check in code to get Tile Filter working with vector images
        else:
            img_comps = []
            for i in range(0, img.GetNumberOfComponentsPerPixel()):
                img_slices_c = [sitk.VectorIndexSelectionCast(s, i)
                                for s in img_slices]
                img_comps.append(sitk.Tile(img_slices_c, [maxlen, d]))
            img = sitk.Compose(img_comps)

    myshow(img, title, margin, dpi)

In [10]:
# 쓰지않는 함수 
def resample(image, new_spacing):
    # Output image Origin, Spacing, Size, Direction 들은 레퍼런스에서 가져옵니다.
    new_size = (512,512,512)
    image = image.SetSpacing(new_spacing[0],new_spacing[1],new_spacing[2])
    transform = sitk.Transform(3, sitk.sitkIdentity)
    interpolator = sitk.sitkCosineWindowedSinc
    resize_factor=np.array(image.GetSpacing())/np.array(new_spacing)
    new_real_shape = np.array(image.GetSize())*resize_factor
    new_shape = np.array(np.round(new_real_shape),np.uint32)
    new_size = (int(new_shape[0]),int(new_shape[1]),int(new_shape[2]))
    origin = image.GetOrigin()
    new_spacing = new_spacing
    direction = image.GetDirection()
    #return sitk.Resample(image, reference_image, transform, interpolator, default_value)
    return sitk.Resample(image,new_size , transform, interpolator, origin, new_spacing, direction)

In [11]:
# resample 함수 
def calculate_origin_offset(new_spacing, old_spacing):
    return np.subtract(new_spacing, old_spacing)/2

def sitk_new_blank_image(size, spacing, direction, origin, default_value=0.):
    image = sitk.GetImageFromArray(np.ones(size, dtype=np.float).T * default_value)
    image.SetSpacing(spacing)
    image.SetDirection(direction)
    image.SetOrigin(origin)
    return image

def sitk_resample_to_image(image, reference_image, default_value=0., interpolator=sitk.sitkLinear, transform=None,
                           output_pixel_type=None):
    if transform is None:
        transform = sitk.Transform()
        transform.SetIdentity()
    if output_pixel_type is None:
        output_pixel_type = image.GetPixelID()
    resample_filter = sitk.ResampleImageFilter()
    resample_filter.SetInterpolator(interpolator)
    resample_filter.SetTransform(transform)
    resample_filter.SetOutputPixelType(output_pixel_type)
    resample_filter.SetDefaultPixelValue(default_value)
    resample_filter.SetReferenceImage(reference_image)
    return resample_filter.Execute(image)

def sitk_resample_to_spacing(image, new_spacing=(1.0, 1.0, 1.0), interpolator=sitk.sitkCosineWindowedSinc, default_value=0.):
    zoom_factor = np.divide(image.GetSpacing(), new_spacing)
    new_size = np.asarray(np.ceil(np.round(np.multiply(zoom_factor, image.GetSize()), decimals=5)), dtype=np.int16)
    offset = calculate_origin_offset(new_spacing, image.GetSpacing())
    reference_image = sitk_new_blank_image(size=new_size, spacing=new_spacing, direction=image.GetDirection(),
                                           origin=image.GetOrigin() + offset, default_value=default_value)
    return sitk_resample_to_image(image, reference_image, interpolator=interpolator, default_value=default_value)


In [12]:
# 이미지 spacing 1.0,1.0,1.0으로 지정후 resample
image_one_spacing = image.SetSpacing((1.0,1.0,1.0))
#test_image = sitk_resample_to_spacing(image,(0.5,0.5,0.5))

In [13]:
# 레이블 spacing 1.0,1.0,1.0 인 처음 상태에서 resample
#test_label = sitk_resample_to_spacing(label,(0.5,0.5,0.5))

In [14]:
# # 이미지 사이즈. 스페이싱 출력
# print(test_image.GetSize())
# print(test_image.GetSpacing())
# print(resampled_img.GetSize())
# print(resampled_img.GetSpacing())
# myshow2d(test_image[:,:,100])
# myshow2d(resampled_img[:,:,100])

In [15]:
# resample 제작한 함수
def resample_POS572(image, new_spacing):
    # Output image Origin, Spacing, Size, Direction 들은 레퍼런스에서 가져옵니다.
    transform = sitk.Transform(3, sitk.sitkIdentity)
    interpolator = sitk.sitkCosineWindowedSinc
    resize_factor=np.array(image.GetSpacing())/np.array(new_spacing)
    new_real_shape = np.array(image.GetSize())*resize_factor
    new_shape = np.array(np.round(new_real_shape),np.uint32)
    new_size = (int(new_shape[0]),int(new_shape[1]),int(new_shape[2]))
    origin = image.GetOrigin()
    new_spacing = new_spacing
    direction = image.GetDirection()
    #return sitk.Resample(image, reference_image, transform, interpolator, default_value)
    return sitk.Resample(image,new_size , transform, interpolator, origin, new_spacing, direction)

In [ ]:
# 이미지데이터 0.5 spacing으로 resample하기
resampled_img = sitk_resample_to_spacing(image,(0.5,0.5,0.5))

In [ ]:
# 레이블 데이터 0.5 spacing으로 resample하기
resampled_label = sitk_resample_to_spacing(label,(0.5,0.5,0.5))

In [ ]:
print(resampled_img.GetSize())
print(resampled_label.GetSize())

In [ ]:
# 이미지 확인 (axial)
myshow2d(image[:,:,50],'image show')


In [ ]:
# resample된 이미지 확인(axial)
myshow2d(resampled_img[:,:,30], 'resampled image show')


In [ ]:
# 레이블 확인(axial)
myshow2d(label[:,:,50],'label show')

In [ ]:
# resample된 레이블 확인(axial)
myshow2d(resampled_label[:,:,100], 'resampled mask show')

In [ ]:
# 512,512 사이즈로 resize해서 resample하기 
def resample_resize2D(image, rc ):
    rows=rc[0]
    cols=rc[1]
    #print(rows,cols)
    newe_size= (rows,cols)
    prior_cols=image.GetWidth()
    prior_rows=image.GetHeight()
    affine = sitk.AffineTransform(2)
    matrix = np.array(affine.GetMatrix()).reshape((2,2))
    matrix[0,0] = float(prior_cols)/cols
    matrix[1,1]= float(prior_rows)/rows
    affine.SetMatrix(matrix.ravel())
    interpolator = sitk.sitkCosineWindowedSinc
    origin = image.GetOrigin()
    new_spacing = image.GetSpacing()
    direction = image.GetDirection()
    #return sitk.Resample(image, new_size , new_transform, interpolator, origin,new_spacing,direction)
    return sitk.Resample(image,(512,512) , affine, interpolator, origin, new_spacing, direction)


In [ ]:
# 512,512 사이즈로 resize해서 resample하기 
def resample_resizeMask2D(image, rc ):
    rows=rc[0]
    cols=rc[1]
    newe_size= (rows,cols)
    prior_cols=image.GetWidth()
    prior_rows=image.GetHeight()
    affine = sitk.AffineTransform(2)
    matrix = np.array(affine.GetMatrix()).reshape((2,2))
    matrix[0,0] = float(prior_cols)/cols
    matrix[1,1]= float(prior_rows)/rows
    affine.SetMatrix(matrix.ravel())
    interpolator = sitk.sitkCosineWindowedSinc
    origin = image.GetOrigin()
    new_spacing = image.GetSpacing()
    direction = image.GetDirection()
    #return sitk.Resample(image, new_size , new_transform, interpolator, origin,new_spacing,direction)
    return sitk.Resample(image,(512,512) , affine, interpolator, origin, new_spacing, direction)

In [ ]:
# class 4개(배경, LVM, LV, RV)로 배열 만들어 sitk로 반환하기
def label_Mask_3ch(image):
    width = image.GetSize()[0]
    height = image.GetSize()[1]
    rgbImage = sitk.LabelToRGB(image)
    npImage = sitk.GetArrayFromImage(rgbImage)
    LV , RV, LVM = np.array((0,0,255)), np.array((0,255,255)),np.array((0,205,0))  
    background,red,green,blue = np.array((1,0,0,0)),np.array((0,1,0,0)),np.array((0,0,1,0)),np.array((0,0,0,1))
    #print(npImage.shape)
    for i in range(0,width):
        for j in range(0,height):
            if RV[1] == npImage[j,i][1]:
                npImage[j,i]= blue
            elif LV[1] == npImage[j,i][1] and LV[2]==npImage[j,i][2]:
                npImage[j,i]= green
            elif LVM[1] == npImage[j,i][1] and LVM[2]==npImage[j,i][2]:
                npImage[j,i]= red
            else:
                npImage[j,i]=background
    new_image = sitk.GetImageFromArray(npImage)
    return new_image
            

In [ ]:
pos572_img_hu = sitk.Cast(sitk.IntensityWindowing(resampled_img,LowT,HighT,0,255),sitk.sitkUInt8)
size3d = pos572_img_hu.GetSize()
size2d = (size3d[0],size3d[1],0)
size3d_label = resampled_label.GetSize()
size2d_label = (size3d_label[0],size3d_label[1],0)
outimg = sitk.Image(resampled_img.GetSize(),sitk.sitkUInt8)
outimg.SetDirection(resampled_img.GetDirection())
outimg.SetSpacing(resampled_img.GetSpacing())
outimg.SetOrigin(resampled_img.GetOrigin())        
outimage_array = sitk.GetArrayFromImage(outimg)
        
num_of_slices = pos572_img_hu.GetDepth()
print(num_of_slices)
test_index = 100
#for i in range(num_of_slices):
for i in range(test_index,test_index+1):
    prev_idx = (0,0,i-displacement)
    cur_idx = (0,0,i)
    next_idx = (0,0,i+displacement)
    
    if(prev_idx[2]<0) or (next_idx[2]>=num_of_slices):
        continue
    
    pre_image = sitk.Extract(pos572_img_hu,size2d,prev_idx)
    cur_image = sitk.Extract(pos572_img_hu,size2d,cur_idx)
    next_image = sitk.Extract(pos572_img_hu,size2d,next_idx)
    
    mask2d = sitk.Extract(resampled_label,size2d_label,cur_idx)
    
    #512 * 512로 추출한 영상들 resize 하기
    prev_image_resample = resample_resize2D(pre_image,(img_rows,img_cols))
    cur_image_resample = resample_resize2D(cur_image,(img_rows,img_cols))
    next_image_resample = resample_resize2D(next_image,(img_rows,img_cols))
    
    mask2d_resample = resample_resizeMask2D(mask2d,(img_rows,img_cols))
    mask_final = label_Mask_3ch(mask2d_resample)
    rgbImage = sitk.Compose(prev_image_resample, cur_image_resample, next_image_resample)
    
    #print(sitk.GetArrayFromImage(prev_image_resample).shape)
    #myshow2d(sitk.LabelToRGB(mask2d_resample))
    myshow2d(mask_final)
    print(pos572_img_hu.GetSize())
    print(resampled_label.GetSize())
    image_train_arr = sitk.GetArrayFromImage(rgbImage)
    label_train_arr = sitk.GetArrayFromImage(mask_final)
    
    image_training = image_train_arr.reshape((1,img_rows,img_cols,3))
    label_training = label_train_arr.reshape((1,img_rows,img_cols,n_class))
    
    myshow2d(cur_image_resample)
    #myshow(sitk.LabelOverlay(cur_image_resample, mask2d_resample), title="Label Overlayed")

    #myshow(sitk.LabelOverlay(prev_image_resample, mask_final), title="Label Overlayed")
    #model.train_on_batch(image_training, label_training)
    
    
    

In [ ]:
compare_index =(test_index*pos572_img_hu.GetDepth())/image.GetDepth()
print(int(compare_index) )
#myshow2d(image[:,:,50])
#myshow2d(label[:,:,50])
#myshow2d(prev_image_resample)
#myshow2d(pos572_img_hu[:,:,40])
#myshow2d(resampled_img[:,:,40])
#myshow2d(image[:,:,50])
#myshow2d(image[:,:,int(compare_index)])
#myshow2d(image[:,:,40])
#myshow2d(label[:,:,40])
print(prev_image_resample.GetSize())

In [ ]:
if len(file_list_image)==len(file_list_label):
    indices_t = len(file_name_image)
    print("The image matches the label.")
else :
    print("Image and label do not match")

In [ ]:
for epoch in range(20):
    random.shuffle(indices_t)
    #random.shuffle(indices_v)
    for n in indices_t:
        
        input_img = sitk.ReadImage(file_name_image[n])
        input_label = sitk.ReadImage(file_name_label[n])
        
        pos572_img = resample_POS572(input_img,(0.5,0.5,0.5))
        pos572_label = resampleMask_POS572(input_label,(0.5,0.5,0.5))
        
        pos572_img_hu = sitk.Cast(sitk.IntensityWindowing(pos572_img,LowT,HighT,0,255),sitk.sitkUInt8)
        
        size3d = pos572_img_hu.GetSize()
        size2d = (size3d[0],size3d[1],0)
        
        outimg = sitk.Image(pos572.GetSize().sitk.sitkUInt8)
        outimg.SetDirection(pos572_img.GetDirection())
        outimg.SetSpacing(pos572_img.GetSpacing())
        outimg.SetOrigin(pos572_img.GetOrigin())
        
        outimage_array = sitk.GetArrayFromImage(outimg)
        
        num_of_slices = pos572_img_hu.GetDepth()